In [1]:
!pip install datasets evaluate 'flwr[simulation]' torch transformers

In [2]:
pip install 'flwr[simulation]'

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install -q trl peft

In [4]:
!pip install transformers

In [5]:
!pip install peft

In [6]:
!pip install evaluate

In [7]:
from collections import OrderedDict
import os
import random
import warnings

import flwr as fl
import torch

from torch.utils.data import DataLoader

from datasets import load_dataset, DatasetDict
from evaluate import load as load_metric
from peft import AdaLoraModel, AdaLoraConfig
from transformers import AutoTokenizer, DataCollatorWithPadding, FalconForSequenceClassification
from transformers import AutoModelForSequenceClassification
from transformers import AdamW
from transformers import logging
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

In [8]:
!mkdir falcon/

mkdir: cannot create directory ‘falcon/’: File exists


In [9]:
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
logging.set_verbosity(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
CHECKPOINT = "albert-base-v2"
DEVICE = torch.device("cuda")
NUM_CLIENTS = 2
NUM_ROUNDS = 8

In [10]:
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
logging.set_verbosity(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter('ignore')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
DEVICE = torch.device("cuda")
NUM_CLIENTS = 2
NUM_ROUNDS = 2

In [11]:
config = AdaLoraConfig(
peft_type="ADALORA", task_type="SEQUENCE_CLASSIFICATION", r=8, lora_alpha=16, target_modules=['query_key_value','dense'],target_r =4,
lora_dropout=0.01,
)

In [12]:
tokenizer = AutoTokenizer.from_pretrained("Rocketknight1/falcon-rw-1b")

In [13]:
model_kwargs = dict(
    use_cache=False,
)

In [14]:
model = FalconForSequenceClassification.from_pretrained("Rocketknight1/falcon-rw-1b", num_labels=2, **model_kwargs)

In [15]:
model = AdaLoraModel(model, config, "default")

# Dataloading

In [16]:
ds = load_dataset("stanfordnlp/sst2")

In [17]:
dataset = DatasetDict({
    "train": ds["train"].shuffle(seed=1000),
    "test": ds["validation"].shuffle(seed=1000)
})

In [18]:
column_names = ds["validation"].column_names

In [19]:
column_names.remove('label')

In [20]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [21]:
def tokenize_function(examples):
        return tokenizer(examples["sentence"], truncation=True,padding='max_length')

In [22]:
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=column_names,)

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

In [23]:
tokenized_dataset['test']

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 872
})

In [24]:
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [25]:
tokenized_dataset["train"] = tokenized_dataset["train"].select(range(25000))

In [26]:
    trainloader = DataLoader(
        tokenized_dataset["train"],
        shuffle=True,
        batch_size=4,
        collate_fn = DataCollatorWithPadding(tokenizer = tokenizer)
    )

    testloader = DataLoader(
        tokenized_dataset["test"], batch_size=4,
        collate_fn = DataCollatorWithPadding(tokenizer = tokenizer)
    )

In [27]:
tokenized_dataset["train"]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

In [28]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [29]:
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2_048


In [30]:
training_args = TrainingArguments(
    output_dir='falcon/',
    # fp16=True,
    # bf16=True,
    do_eval=True,
    eval_strategy="epoch",
    gradient_accumulation_steps=128,  # Accumulate gradients and perform parameter updating to conserve memory usage
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    # log_level="info",   # print the number of trainable params, info of saving and model when training
    logging_steps=1,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=2,
    num_train_epochs=1,  ### donot give max steps if you want config from num_train_epochs
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    report_to="tensorboard",
    seed=42,
    save_strategy="epoch",
    save_total_limit=3,
    label_names=["labels"],
    remove_unused_columns=False,
)

In [31]:
trainer = Trainer(
    model=model,
    # tokenizer=tokenizer,
    args=training_args,
    # packing=True,   # see tokenizer.max_model_length
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    # dataset_text_field="sentence",


    # peft_config=peft_config,
    # model_init_kwargs=model_kwargs,   # pass if you want to load from trainer
    # max_seq_length=tokenizer.model_max_length
)

In [32]:
model.to('cuda')

AdaLoraModel(
  (model): FalconForSequenceClassification(
    (transformer): FalconModel(
      (word_embeddings): Embedding(50304, 2048)
      (h): ModuleList(
        (0-23): 24 x FalconDecoderLayer(
          (self_attention): FalconAttention(
            (query_key_value): adalora.SVDLinear(
              (base_layer): FalconLinear(in_features=2048, out_features=6144, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.01, inplace=False)
              )
              (lora_A): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 12x2048 (cuda:0)])
              (lora_B): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 6144x12 (cuda:0)])
              (lora_embedding_A): ParameterDict()
              (lora_embedding_B): ParameterDict()
              (lora_E): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 12x1 (cuda:0)])
              (ranknum): Para

In [33]:
def train(net, trainloader, epochs):
    optimizer = AdamW(net.parameters(), lr=5e-5)
    net.train()
    for _ in range(epochs):
        for batch in trainloader:
            batch = {k: v.to(DEVICE) for k, v in batch.items()}
            outputs = net(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

In [34]:
def test(net, testloader):
    metric = load_metric("accuracy")
    loss = 0
    net.eval()
    for batch in testloader:
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        with torch.no_grad():
            outputs = net(**batch)
        logits = outputs.logits
        loss += outputs.loss.item()
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])
    loss /= len(testloader.dataset)
    accuracy = metric.compute()["accuracy"]
    return loss, accuracy

In [35]:
class SSTClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, testloader):
        self.net = net
        self.trainloader = trainloader
        self.testloader = testloader

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.net.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
        self.net.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        print("Training Started...")
        train(self.net, self.trainloader, epochs=5)
        print("Training Finished.")
        return self.get_parameters(config={}), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        loss, accuracy = test(self.net, self.testloader)
        return float(loss), len(self.testloader), {"accuracy": float(accuracy), "loss": float(loss)}

In [36]:
def client_fn(cid):
  return SSTClient(model, trainloader, testloader)

In [37]:
from accelerate import Accelerator

accelerator = Accelerator()

In [38]:
def weighted_average(metrics):
  accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
  losses = [num_examples * m["loss"] for num_examples, m in metrics]
  examples = [num_examples for num_examples, _ in metrics]
  return {"accuracy": sum(accuracies) / sum(examples), "loss": sum(losses) / sum(examples)}

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1,
    fraction_evaluate=1,
    evaluate_metrics_aggregation_fn=weighted_average,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1,
                     },
    ray_init_args={"log_to_driver": False, "num_cpus": 1, "num_gpus": 1}
)

INFO :      Starting Flower simulation, config: num_rounds=2, no round_timeout
2024-07-18 09:23:07,633	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 1.0, 'object_store_memory': 19135523635.0, 'memory': 38271047271.0, 'node:10.192.12.37': 1.0, 'GPU': 1.0, 'node:__internal_head__': 1.0, 'accelerator_type:L4': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
INFO :      Received initial parameters from one random client
INFO :      Evaluating initial global parameters
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
ERROR :     Traceback (most recent cal

In [29]:
torch.save(model, 'Falcon_model.pt')

# New_Model

AlBerta Tokenizer

In [24]:
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(
    CHECKPOINT, num_labels=2
)

In [15]:
model

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [16]:
AlbertaConfig = AdaLoraConfig(
peft_type="ADALORA", task_type="SEQUENCE_CLASSIFICATION", r=8, lora_alpha=16, target_modules=['query','key','value'],target_r =4,
lora_dropout=0.01,
)

In [45]:
    trainloader = DataLoader(
        tokenized_dataset["train"],
        shuffle=True,
        batch_size=32,
        collate_fn = DataCollatorWithPadding(tokenizer = tokenizer)
    )

    testloader = DataLoader(
        tokenized_dataset["test"], batch_size=32,
        collate_fn = DataCollatorWithPadding(tokenizer = tokenizer)
    )

In [38]:
model = AdaLoraModel(model, AlbertaConfig, "default").to(DEVICE)

In [46]:
def train(net, trainloader, epochs):
    optimizer = AdamW(net.parameters(), lr=5e-5)
    net.train()
    for _ in range(epochs):
        for batch in trainloader:
            batch = {k: v.to(DEVICE) for k, v in batch.items()}
            outputs = net(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()


def test(net, testloader):
    metric = load_metric("accuracy")
    loss = 0
    net.eval()
    for batch in testloader:
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        with torch.no_grad():
            outputs = net(**batch)
        logits = outputs.logits
        loss += outputs.loss.item()
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])
    loss /= len(testloader.dataset)
    accuracy = metric.compute()["accuracy"]
    return loss, accuracy

In [52]:
class IMDBClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, testloader):
        self.net = net
        self.trainloader = trainloader
        self.testloader = testloader

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.net.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
        self.net.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        print("Training Started...")
        train(self.net, self.trainloader, epochs=5)
        print("Training Finished.")
        return self.get_parameters(config={}), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        loss, accuracy = test(self.net, self.testloader)
        return float(loss), len(self.testloader), {"accuracy": float(accuracy), "loss": float(loss)}

In [53]:
def client_fn(cid):
  return IMDBClient(model, trainloader, testloader)

In [54]:
def weighted_average(metrics):
  accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
  losses = [num_examples * m["loss"] for num_examples, m in metrics]
  examples = [num_examples for num_examples, _ in metrics]
  return {"accuracy": sum(accuracies) / sum(examples), "loss": sum(losses) / sum(examples)}

strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=1.0,
    evaluate_metrics_aggregation_fn=weighted_average,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
    client_resources={"num_cpus": 1, "num_gpus": 1},
    ray_init_args={"log_to_driver": False, "num_cpus": 1, "num_gpus": 1}
)

INFO :      Starting Flower simulation, config: num_rounds=8, no round_timeout
2024-07-17 18:45:04,384	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 18994330828.0, 'memory': 37988661659.0, 'GPU': 1.0, 'node:10.192.10.193': 1.0, 'node:__internal_head__': 1.0, 'accelerator_type:L4': 1.0, 'CPU': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
INFO :      Received initial parameters from one random client
INFO :      Evaluating initial global parameters
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
INFO :      aggregate_fit: received 2

History (loss, distributed):
	round 1: 0.023641480765211473
	round 2: 0.02160511651170363
	round 3: 0.019594239566577683
	round 4: 0.010389233076381028
	round 5: 0.008950578945970863
	round 6: 0.007895996956445208
	round 7: 0.009031598968858566
	round 8: 0.008141585540265665
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.5286697247706422),
              (2, 0.5344036697247706),
              (3, 0.7522935779816514),
              (4, 0.8899082568807339),
              (5, 0.9059633027522935),
              (6, 0.9128440366972477),
              (7, 0.9025229357798165),
              (8, 0.9071100917431193)],
 'loss': [(1, 0.023641480765211473),
          (2, 0.02160511651170363),
          (3, 0.019594239566577683),
          (4, 0.010389233076381028),
          (5, 0.008950578945970863),
          (6, 0.007895996956445208),
          (7, 0.009031598968858566),
          (8, 0.008141585540265665)]}

In [ ]:
torch.save(model, 'Alberta_model.pth')